In [4]:
import yaml
import pyodbc
from market_growth_analysis.etl.stagging import *

# Load the YAML file
with open('../../conf/local.yml', 'r') as f:
    config = yaml.safe_load(f)


In [5]:
stagging_db_conn = config['stagging_db']

In [6]:
# Extract the connection information
driver = stagging_db_conn['driver']
server = stagging_db_conn['server']
database = stagging_db_conn['database']
trusted_connection = stagging_db_conn['trusted_connection']

# Create the connection string
connection_string = f"Driver={{{driver}}};Server={server};Database={database};Trusted_Connection={trusted_connection};"
connection = pyodbc.connect(connection_string)

# Create a cursor object
cursor = connection.cursor()

In [7]:
# CSV files to load
csv_files = [
    "balance-sheet.csv",
    "cash-flow-statement.csv",
    "financial-ratios.csv",
    "income-statement.csv",
    "prices.csv"
]


In [8]:
create_stg_balance_sheet_statement(connection, 'balance-sheet')
create_stg_cash_flow_statement(connection, 'cash-flow-statement')
create_stg_financial_ratios(connection, 'financial-ratios')
create_stg_income_statement(connection, 'income-statement')
create_stg_prcies(connection, 'prcies')

In [18]:
# Drop existing tables
for csv_file in csv_files:
    table_name = csv_file.replace(".csv", "")
    drop_table_query = f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE [{table_name}]"
    cursor.execute(drop_table_query)
    connection.commit()


In [40]:
# Load CSV files into the database
for csv_file in csv_files:
    table_name = csv_file.replace(".csv", "")
    file_path = r"../../data/raw_01/" + csv_file  # Use raw string (r) to preserve backslashes

    # Prepare the BULK INSERT statement
    bulk_insert_query = f"""

        -- truncate the table first
        TRUNCATE TABLE dbo.['{table_name}'];
        GO

        -- import the file
        BULK INSERT dbo.['{table_name}']
        FROM '{file_path}'
        WITH (
            FORMAT = 'CSV',
            FIRSTROW = 2 
        );
        GO
    """

    # Execute the BULK INSERT statement
    cursor.execute(bulk_insert_query)
    connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()



ProgrammingError: ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Sintaxis incorrecta cerca de 'GO'. (102) (SQLExecDirectW); [42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Sintaxis incorrecta cerca de 'GO'. (102)")